In [192]:
# ==== Term Frequency Calculation by jieba ====
# 1.--- load dictionary and stop_words 
import jieba
# remove punctuation


jieba.set_dictionary('C:/Users/doudi/Downloads/dict.txt.big')
# jieba.load_userdict('')

# load stop word file
stw=[]
with open('C:/Users/doudi/Downloads/chineses_stop_word.txt', 'r') as file:
    for data in file.readlines():
        data = data.strip()
        stw.append(data)

In [194]:
# 2.--- tf calculator
import re
import string
import zhon.hanzi
import jieba.analyse

def open_txt(path1, st_word):
    seg=[]
    seg_result=[]
    with open(path1, 'r', encoding='UTF-8') as file:
        text = file.read()      
            
        # remove what is not in Chinese characters
        r = re.compile(u'[^\u4e00-\u9fa5]')
        text = r.sub(r'',text)
        
        # corpus
        seg = jieba.cut(text, cut_all=False)
        seg_result = list(filter(lambda a: a not in st_word, seg))
        
        
        # write a file to see corpus outcome
        with open('C:/Users/doudi/Downloads/偵錯.txt', 'w',encoding='utf-8') as out_file:
            out_file.write("\n".join(seg_result))  
            
        # calculate the accumulated term numbers : Method【1】
       
        from collections import Counter
        c = Counter()
        for x in seg_result:
            if len(x)>1 and x != '\r\n':
                c[x] += 1
        for (k,v) in c.most_common(30):# 输出词频最高的前两个词
            print("%s:%d"%(k,v))
        
        return c
        
        '''
        
        # calculate the accumulated term numbers : Method【2】
        counts = {}  
        for word in seg_result:  
            counts[word] = counts.get(word,0) + 1
        items = list(counts.items())  
        items.sort(key=lambda x:x[1], reverse=True) # [(x,1),(y,2)]依照1,2排序
        
        for l in range(30):  
            word, count = items[l]  
            print ("{0:<10}{1:>5}".format(word, count))
        
        # calculate term weight in order to plot
        # keywords = jieba.analyse.extract_tags(text, topK=20, withWeight=True)
        
        
        return counts
        # write file
        '''
p_1 = 'C:/Users/doudi/Downloads/ChineseDataset_Assignment1.txt'

dict_tf = open_txt(p_1, stw)

林書豪:12592
尼克:8806
台北:6419
學校:6042
政府:5141
美國:5009
比賽:4973
總統:4907
大學:4667
立委:4431
老師:4262
民進黨:4253
教育:4156
不過:4072
孩子:3791
中國:3670
國民黨:3668
時間:3447
一個:3393
校長:3381
高中:3362
最後:3332
家長:3316
參加:3312
活動:3252
進行:3242
表現:3199
未來:3150
中央社:3018
球員:3009


In [195]:
# 3.--- preparing the target data to plot
import csv

# save tf 
w = csv.writer(open("C:/Users/doudi/Downloads/term_freq.csv", "w", newline='', encoding = 'UTF-8'))
for key, val in dict_tf.items():
    w.writerow([key, val])

# sort the dict by value
import operator

tf = sorted(dict_tf.items(), key=operator.itemgetter(1), reverse=True)
print(tf[0:5])

[('林書豪', 12592), ('尼克', 8806), ('台北', 6419), ('學校', 6042), ('政府', 5141)]


In [196]:
# 4. --- plot the Zipf's graph (ranking 30)
ext30_tf = tf[0:30]

import plotly.graph_objects as go
x = list(range(1,31))
y = []

n = 1 
temp = [m[n] for m in ext30_tf]

for i in range(len(temp)):
    y.append((temp[i]/sum(temp)))

fig = go.Figure(go.Scatter(x=x, y=y, mode='lines+markers', name='lines+markers'))
fig.update_layout(title='Zipf‘s law chart (ranking 30)',xaxis_title='Ranking',yaxis_title='Term weight')
fig.show()

In [197]:
# 4-1. --- plot the Zipf's graph (ranking 100)
ext100_tf = tf[0:100]

x1 = list(range(1,101))
y1 = []

n = 1 
temp1 = [m[n] for m in ext100_tf]

for i in range(len(temp1)):
    y1.append((temp1[i]/sum(temp1)))

fig = go.Figure(go.Scatter(x=x1, y=y1, mode='lines+markers', name='lines+markers'))
fig.update_layout(title='Zipf‘s law chart (ranking 100)',xaxis_title='Ranking',yaxis_title='Term weight')
fig.show()

In [198]:
# ==== TF-IDF Calculation by SKlearn ====
# Cleaning 
def Cleaning(line):
    r = re.compile(u'[^\u4e00-\u9fa5]')
    line = r.sub(r'',line)
    return line
# load data
f = []
with open('C:/Users/doudi/Downloads/ChineseDataset_Assignment1.txt', 'r', encoding='utf-8') as file:
    for l in file: 
        l = l.strip()
        l = Cleaning(l)
        f.append(" ".join(jieba.cut(l, cut_all=False)))
    

In [199]:
# Bag-of-Words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
training = vectorizer.fit_transform(f)

training.shape

(15000, 158883)

In [200]:
# TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

count_vec = TfidfVectorizer(decode_error='ignore', max_features=20, stop_words=stw)
feature_matrix = count_vec.fit_transform(f)

# first doc
first_vector_tfidf = feature_matrix[0]
 
# place tf-idf values in a pandas data frame
df = pd.DataFrame(first_vector_tfidf.T.todense(), index=count_vec.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
尼克,0.731520
林書豪,0.675649
比賽,0.091527
一個,0.000000
時間,0.000000
美國,0.000000
總統,0.000000
立委,0.000000
民進黨,0.000000
校長,0.000000


In [201]:
terms = count_vec.get_feature_names()

# sum tfidf frequency of each term through documents
sums = feature_matrix.sum(axis=0)

# connecting term to its sums frequency
data = []
for col, term in enumerate(terms):
    data.append( (term, sums[0,col] ))

ranking = pd.DataFrame(data, columns=['term','rank'])
print(ranking.sort_values('rank', ascending=False))

   term         rank
3    台北  1700.256853
12  林書豪  1495.525624
14   比賽  1321.633241
7    學校  1197.018662
9    政府  1142.020934
1    不過  1135.509751
18   美國  1104.046386
8    尼克  1080.546575
5    大學  1023.042479
17   總統   983.845311
11   時間   936.389331
0    一個   865.862087
16   立委   852.553587
19   老師   850.860817
10   教育   762.816516
15  民進黨   734.637771
6    孩子   730.205074
2    中國   725.755513
4   國民黨   665.308279
13   校長   633.046419
